# IMPORT REQUIRED LIBRARIES

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
import numpy as np
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
import torch.optim as optim
from PIL import Image
import time

# LOAD DATASET AND NORMALIZE

In [2]:
mean, std = 0.1307, 0.3081

train_dataset = MNIST('./Datasets/MNIST', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
test_dataset = MNIST('./Datasets/MNIST', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                            ]))

# TRIPLET LOSS FUNCTION

In [3]:
class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
    def forward(self, anchor, positive, negative, size_average=True):
        distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
        distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean() if size_average else losses.sum()


# CUSTOMIZE MNIST TO BE TRIPLETS

In [4]:
class TripletMNIST(Dataset):
    def __init__(self, mnist_dataset):
        self.mnist_dataset = mnist_dataset
        self.train = self.mnist_dataset.train
        self.transform = self.mnist_dataset.transform

        if self.train:
            self.train_labels = self.mnist_dataset.train_labels
            self.train_data = self.mnist_dataset.train_data
            self.labels_set = set(self.train_labels.numpy())
            self.label_to_indices = {label: np.where(self.train_labels.numpy() == label)[0]
                                     for label in self.labels_set}

        else:
            self.test_labels = self.mnist_dataset.test_labels
            self.test_data = self.mnist_dataset.test_data
            # generate fixed triplets for testing
            self.labels_set = set(self.test_labels.numpy())
            self.label_to_indices = {label: np.where(self.test_labels.numpy() == label)[0]
                                     for label in self.labels_set}

            random_state = np.random.RandomState(29)

            triplets = [[i,
                         random_state.choice(self.label_to_indices[self.test_labels[i].item()]),
                         random_state.choice(self.label_to_indices[
                                                 np.random.choice(
                                                     list(self.labels_set - set([self.test_labels[i].item()]))
                                                 )
                                             ])
                         ]
                        for i in range(len(self.test_data))]
            self.test_triplets = triplets

    def __getitem__(self, index):
        if self.train:
            img1, label1 = self.train_data[index], self.train_labels[index].item()
            positive_index = index
            while positive_index == index:
                positive_index = np.random.choice(self.label_to_indices[label1])
            negative_label = np.random.choice(list(self.labels_set - set([label1])))
            negative_index = np.random.choice(self.label_to_indices[negative_label])
            img2 = self.train_data[positive_index]
            img3 = self.train_data[negative_index]
        else:
            img1 = self.test_data[self.test_triplets[index][0]]
            img2 = self.test_data[self.test_triplets[index][1]]
            img3 = self.test_data[self.test_triplets[index][2]]

        img1 = Image.fromarray(img1.numpy(), mode='L')
        img2 = Image.fromarray(img2.numpy(), mode='L')
        img3 = Image.fromarray(img3.numpy(), mode='L')
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)
        return (img1, img2, img3), []

    def __len__(self):
        return len(self.mnist_dataset)


# NEWORK ARCHITECTURE

In [5]:
class EmbeddingNet(nn.Module):
    def __init__(self):
        super(EmbeddingNet, self).__init__()
        self.convnet = nn.Sequential(nn.Conv2d(1, 32, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2),
                                     nn.Conv2d(32, 64, 5), nn.PReLU(),
                                     nn.MaxPool2d(2, stride=2))

        self.fc = nn.Sequential(nn.Linear(64 * 4 * 4, 256),
                                nn.PReLU(),
                                nn.Linear(256, 256),
                                nn.PReLU(),
                                nn.Linear(256, 2)
                                )

    def forward(self, x):
        output = self.convnet(x)
        output = output.view(output.size()[0], -1)
        output = self.fc(output)
        return output

    def get_embedding(self, x):
        return self.forward(x)

# TRIPLET WRAPPER

In [6]:
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, x1, x2=None, x3=None):
        if x2 is None and x3 is None:
            return self.embedding_net(x1)
        return self.embedding_net(x1),self.embedding_net(x2),self.embedding_net(x3)

    def get_embedding(self, x):
        return self.embedding_net(x)

# SETUPS

In [8]:
cuda = torch.cuda.is_available()
triplet_train_dataset = TripletMNIST(train_dataset) 
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset,
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  **kwargs)
margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

# TRAIN

In [10]:
def fit(train_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[],
        start_epoch=0):
    for epoch in range(0, start_epoch):
        scheduler.step()

    for epoch in range(start_epoch, n_epochs):
        scheduler.step()
        train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)
        message = 'Epoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch + 1, n_epochs, train_loss)
        for metric in metrics:
            message += '\t{}: {}'.format(metric.name(), metric.value())
        print(message)


def train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics):
    for metric in metrics:
        metric.reset()

    model.train()
    losses = []
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        target = target if len(target) > 0 else None
        if not type(data) in (tuple, list):
            data = (data,)
        if cuda:
            data = tuple(d.cuda() for d in data)
            if target is not None:
                target = target.cuda()


        optimizer.zero_grad()
        outputs = model(*data)
        print(outputs)
        if type(outputs) not in (tuple, list):
            outputs = (outputs,)

        loss_inputs = outputs
        if target is not None:
            target = (target,)
            loss_inputs += target

        loss_outputs = loss_fn(*loss_inputs)
        loss = loss_outputs[0] if type(loss_outputs) in (tuple, list) else loss_outputs
        losses.append(loss.item())
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for metric in metrics:
            metric(outputs, target, loss_outputs)

        if batch_idx % log_interval == 0:
            message = 'Train: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data[0]), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), np.mean(losses))
            for metric in metrics:
                message += '\t{}: {}'.format(metric.name(), metric.value())

            print(message)
            losses = []

    total_loss /= (batch_idx + 1)
    return total_loss, metrics

In [11]:
fit(triplet_train_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/home/ari-18308/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(tensor([[ 1.0531e-01,  3.4335e-03],
        [ 1.3939e-01, -2.1750e-02],
        [ 1.2765e-01, -7.6256e-03],
        [ 1.2166e-01,  2.2495e-02],
        [ 1.2138e-01, -2.0838e-02],
        [ 1.1317e-01, -4.8914e-03],
        [ 1.6028e-01,  1.8080e-02],
        [ 1.6362e-01, -3.2913e-04],
        [ 1.1883e-01,  7.8362e-03],
        [ 1.4764e-01, -8.3395e-03],
        [ 1.4234e-01, -2.3863e-02],
        [ 1.3170e-01, -1.4658e-02],
        [ 9.3261e-02, -2.7425e-02],
        [ 1.4065e-01,  1.9992e-02],
        [ 1.4056e-01, -1.2661e-02],
        [ 1.0236e-01, -7.7702e-03],
        [ 1.4352e-01, -6.7864e-03],
        [ 1.2343e-01,  1.9241e-03],
        [ 1.4363e-01, -6.5379e-03],
        [ 1.4508e-01, -1.8486e-02],
        [ 1.3632e-01, -2.0281e-02],
        [ 1.1683e-01, -2.2008e-02],
        [ 1.2362e-01, -7.3279e-03],
        [ 1.3669e-01, -1.5354e-02],
        [ 1.4627e-01,  2.1785e-02],
        [ 1.1324e-01,  2.2321e-03],
        [ 9.5048e-02, -1.6395e-02],
        [ 1.0324e-01, -1.45

(tensor([[ 0.4219,  0.1137],
        [ 0.1739,  0.0478],
        [ 0.9358, -0.5873],
        [ 0.3841,  0.2284],
        [ 0.9660, -0.6408],
        [ 0.7597, -0.6004],
        [ 0.9041, -0.7323],
        [ 0.4141,  0.2167],
        [ 0.1409,  0.3451],
        [ 0.5795,  0.0287],
        [ 0.1423,  0.2668],
        [ 0.6176, -0.2888],
        [ 0.6977, -0.5762],
        [ 0.4338, -0.2806],
        [ 0.3588,  0.0344],
        [ 0.7611, -0.1310],
        [ 0.4937, -0.0872],
        [ 0.9056, -0.8861],
        [ 0.5704, -0.1692],
        [ 0.3514, -0.0852],
        [ 0.5443, -0.0234],
        [ 0.4358,  0.1272],
        [ 0.7088, -0.5388],
        [ 0.4707,  0.0578],
        [ 0.4676,  0.0759],
        [ 0.5175,  0.1918],
        [ 0.9420, -0.4344],
        [ 0.5474,  0.1042],
        [ 0.4104,  0.2449],
        [ 0.3332,  0.1584],
        [ 0.2800,  0.0736],
        [ 0.8570, -0.7227],
        [ 0.2259, -0.0114],
        [ 0.1603,  0.0777],
        [ 0.7839, -0.2970],
        [ 0.3271,  

(tensor([[ 1.6842, -0.4428],
        [ 1.1760, -0.8451],
        [ 1.0182, -0.1995],
        [ 0.9178, -0.3345],
        [ 2.5919, -2.2376],
        [ 1.0611,  0.0719],
        [ 1.2087,  0.0727],
        [ 1.4727, -0.4693],
        [ 1.0140, -0.0666],
        [ 0.9339,  0.5172],
        [ 2.3276, -1.7123],
        [ 0.9057, -0.0308],
        [ 1.7843, -0.2610],
        [ 1.5741,  0.0357],
        [ 1.6130, -0.8861],
        [ 0.7157,  0.5916],
        [ 1.6190,  0.1802],
        [ 0.9479, -0.5785],
        [ 0.7080,  1.0160],
        [ 1.5141, -0.4500],
        [ 1.3960, -0.5766],
        [ 1.7631, -0.4539],
        [ 1.5943, -0.8757],
        [ 0.8331,  0.0503],
        [ 0.7943,  0.1530],
        [ 1.2576, -0.1999],
        [ 1.4271, -0.3063],
        [ 1.5109, -0.0616],
        [ 0.8560, -0.1363],
        [ 1.0282,  0.3048],
        [ 2.2441, -2.0940],
        [ 1.4895,  0.0102],
        [ 2.1205, -1.1175],
        [ 2.2115, -1.0056],
        [ 1.2050,  0.2276],
        [ 1.6072, -

(tensor([[ 1.0521e+00,  7.7424e-02],
        [ 2.1949e+00, -8.5956e-02],
        [ 2.3716e+00,  5.3217e-01],
        [ 1.6955e+00,  7.7503e-01],
        [ 2.5021e+00, -3.5015e-01],
        [ 2.1106e+00,  4.7449e-01],
        [ 1.6977e+00,  9.6784e-02],
        [ 2.4783e+00,  1.5747e-01],
        [ 9.5484e-01,  4.0203e-01],
        [ 1.9530e+00,  2.2946e-01],
        [ 1.0820e+00,  7.2085e-01],
        [ 2.1703e+00, -7.9241e-01],
        [ 1.5686e+00,  6.2575e-01],
        [ 2.7283e+00, -2.2294e-01],
        [ 1.6808e+00,  3.0098e-01],
        [ 2.6647e+00, -3.1108e-01],
        [ 1.5536e+00,  7.5742e-01],
        [ 2.2050e+00, -2.7334e-01],
        [ 2.4540e+00,  5.2346e-01],
        [ 1.2718e+00,  4.0579e-01],
        [ 1.2412e+00,  7.0682e-01],
        [ 2.4589e+00, -3.0087e-01],
        [ 1.4972e+00,  3.4243e-01],
        [ 2.4492e+00,  6.8682e-01],
        [ 1.1075e+00,  1.1419e+00],
        [ 1.0491e+00,  8.5051e-01],
        [ 2.2798e+00, -1.3726e-01],
        [ 1.8875e+00,  1.07

...

(tensor([[-0.0414,  0.9221],
        [ 0.2461, -0.0084],
        [ 0.1714,  1.4997],
        [ 1.7162,  1.0479],
        [-0.4121,  1.0611],
        [-0.0485, -0.5274],
        [ 1.1911,  0.7421],
        [ 1.8598,  1.0834],
        [ 0.9820,  1.1537],
        [-0.0171, -0.7650],
        [ 2.0487,  1.2036],
        [ 1.0167, -0.8623],
        [ 0.3291,  1.6315],
        [ 1.5775,  0.7781],
        [ 1.8849,  0.6413],
        [ 2.0988,  1.9183],
        [ 0.1580, -0.2836],
        [ 2.3501, -0.4012],
        [ 1.5670, -0.5297],
        [ 1.9806,  0.4589],
        [ 1.9702,  0.8658],
        [ 1.0109,  1.3847],
        [ 0.6711,  0.8958],
        [ 1.8156,  0.4682],
        [ 1.8651,  0.4117],
        [ 0.0902, -0.5676],
        [-0.1127,  0.6727],
        [ 1.3995,  1.2762],
        [ 2.6486,  0.8737],
        [ 1.9664, -0.2468],
        [-0.4598,  0.5332],
        [ 0.2656, -0.5154],
        [ 0.1703,  0.1897],
        [ 0.3086, -0.2110],
        [ 1.7213,  0.6958],
        [ 1.6154,  

(tensor([[ 1.0302e-01,  1.8668e+00],
        [ 1.8138e+00,  1.3197e+00],
        [ 1.8346e+00, -2.9727e-02],
        [ 1.6159e+00,  1.4331e+00],
        [ 2.2608e+00,  6.9882e-01],
        [ 1.4748e+00,  1.0449e+00],
        [ 1.1994e+00, -9.5435e-01],
        [-4.2451e-02, -2.8970e-01],
        [ 1.6919e+00,  2.1615e-01],
        [ 3.4426e-01,  1.2866e+00],
        [ 1.7109e+00, -1.3080e-01],
        [ 1.4151e+00, -8.7081e-01],
        [-5.2855e-01,  7.7745e-01],
        [ 2.3068e+00, -7.6796e-01],
        [ 1.5089e+00, -6.2531e-01],
        [ 9.5240e-01, -7.1316e-01],
        [ 3.0807e-01,  1.7237e+00],
        [ 1.4979e+00,  3.5975e-01],
        [ 1.0096e+00, -8.8180e-02],
        [ 5.1257e-01, -2.5606e-01],
        [ 2.1641e+00,  1.1827e+00],
        [-1.1920e-01, -5.8705e-01],
        [ 2.1286e+00,  2.6879e-01],
        [ 2.6215e-02,  1.8702e-03],
        [ 1.6272e+00,  8.6978e-01],
        [ 9.8873e-01,  1.1417e+00],
        [ 2.4681e+00, -4.8886e-01],
        [ 1.0736e-02,  1.85

(tensor([[ 1.4003e+00,  4.0661e-01],
        [ 4.7262e-02, -1.1069e+00],
        [ 1.8193e+00, -6.5243e-02],
        [ 1.2101e+00,  5.8365e-01],
        [ 5.4504e-01, -7.9223e-01],
        [ 9.0970e-01, -6.7662e-01],
        [-1.6758e-01,  1.3198e+00],
        [-2.2457e-01,  1.3667e+00],
        [ 5.2266e-01,  1.5608e+00],
        [ 1.1222e+00, -1.2942e+00],
        [ 1.8369e+00,  6.4123e-03],
        [ 4.1618e-01,  1.7080e+00],
        [ 1.4236e+00,  5.9172e-01],
        [ 1.7444e+00,  1.2944e+00],
        [ 2.8839e-01, -2.4603e-01],
        [ 2.2643e-01,  8.4461e-01],
        [ 1.6243e+00,  1.5367e+00],
        [ 1.1688e+00,  6.7557e-01],
        [ 1.9989e+00,  5.1620e-01],
        [ 8.4201e-01,  2.6607e-01],
        [ 1.7498e+00, -1.8763e-02],
        [-9.7491e-02,  1.7348e+00],
        [ 1.8334e+00,  1.6655e-02],
        [ 4.5977e-01,  1.4963e+00],
        [-1.2035e-01,  9.3127e-01],
        [ 5.0060e-03, -9.2007e-01],
        [ 1.5012e+00,  4.7330e-01],
        [ 1.9056e+00, -8.70

(tensor([[ 0.0233,  1.0811],
        [-0.2558,  1.7010],
        [-0.2604,  1.5783],
        [ 1.4038,  0.5785],
        [ 0.7118, -1.3997],
        [ 0.9829,  1.6960],
        [-0.1252,  1.7875],
        [-0.6377,  1.6717],
        [ 1.3069, -0.7336],
        [ 1.2582, -1.1119],
        [ 1.0780,  1.8667],
        [-0.1118,  0.9011],
        [ 1.3406,  0.5843],
        [-0.8716,  1.1090],
        [-0.2247,  2.0351],
        [ 0.2024, -1.3534],
        [-0.3683, -0.7832],
        [-0.2285,  0.6499],
        [ 0.1940,  0.5142],
        [-0.4167,  1.1419],
        [ 1.3639,  0.4624],
        [-0.0899, -0.1974],
        [-0.3457,  1.6488],
        [-0.1612,  1.8618],
        [-0.7190,  1.3734],
        [-0.9577,  1.5314],
        [-0.3403,  0.0995],
        [ 0.5699, -1.0557],
        [ 1.1582,  0.6232],
        [-0.7701,  1.6438],
        [-0.3904,  1.5165],
        [ 0.6811,  0.7019],
        [ 1.6746,  0.6362],
        [ 0.8747,  0.6946],
        [-0.6779,  1.4187],
        [ 1.9643,  

KeyboardInterrupt: 

# VALIDATE

In [16]:
torch.save(model,"Models/tripletMNIST.pt")

In [17]:
model_loaded = torch.load("Models/tripletMNIST.pt")

In [18]:
import torch
import torch.nn.functional as F

def evaluate_model(model, triplet_test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        start = time.time()
        for (anchor, positive, negative),[] in triplet_test_loader:
            anchor_embedding,positive_embedding,negative_embedding = model(anchor,positive,negative)
            positive_distance = F.pairwise_distance(anchor_embedding, positive_embedding)
            negative_distance = F.pairwise_distance(anchor_embedding, negative_embedding)
            correct += torch.sum(positive_distance < negative_distance).item()
            total += anchor.size(0)
    accuracy = correct / total
    print('Accuracy : {:.2f}%\nTime : {:.2f} SECONDS'.format(accuracy * 100,time.time()-start))

In [19]:
evaluate_model(model_loaded,triplet_test_loader)

Accuracy : 99.67%
Time : 7.93 SECONDS


# WITH FACEBOOK AI SIMILARITY SEARCH

In [20]:
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, **kwargs)
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **kwargs)

In [25]:
import faiss

In [26]:
embs1 = None
labels1 = []
for idx,i in enumerate(train_loader):
    if idx==10000: break
    I, L = i
    labels1.append(L)
    emb = model_loaded(I) # Assuming `model_loaded(I)` returns a PyTorch tensor
    emb = emb.detach()
    if embs1 is None:
        embs1 = emb
    else:
        embs1 = torch.cat((embs1, emb), dim=0)

In [27]:
embs2 = None
labels2 = []
for i in test_loader:
    I, L = i
    labels2.append(L)
    emb = model_loaded(I)
    if embs2 is None:
        embs2 = emb
    else:
        embs2 = torch.cat((embs2, emb), dim=0)

In [29]:
embs = embs1

In [44]:
index1 = faiss.IndexFlatL2(embs.shape[1])  # Assuming embs.shape[1] represents the dimensionality of the embeddings
index1.add(embs)

nlist = 100  # Number of cells/buckets
quantizer = faiss.IndexFlatL2(embs.shape[1])  # Quantizer index (same as IndexFlatL2)
index2 = faiss.IndexIVFFlat(quantizer, embs.shape[1], nlist)
index2.train(embs)
index2.add(embs)

index3 = faiss.IndexHNSWFlat(embs.shape[1], 32)  # M = 32 for the HNSW index
index3.add(embs)

nbits = 8  # Number of bits for the LSH hash
index4 = faiss.IndexLSH(embs.shape[1], nbits)
index4.add(embs)


# EVALUATE WITH FAISS

In [31]:
def evaluatewithfaiss(embs,index):
    TOTAL = len(embs)
    CORRECT = 0
    start = time.time()
    for idx,emb in enumerate(embs):
        label = index.search(emb.detach().reshape(1,-1),1)[1][0][0]
        CORRECT += labels1[label]==labels2[idx]
    return (CORRECT/TOTAL*100).item(),f'{time.time()-start} SECONDS'
        

In [32]:
print(f'IndexFlatL2 : {evaluatewithfaiss(embs2,index1)}')
print(f'IndexIVFFlat : {evaluatewithfaiss(embs2,index2)}')
print(f'IndexHNSWFlat : {evaluatewithfaiss(embs2,index3)}')
print(f'IndexLSH : {evaluatewithfaiss(embs2,index4)}')

IndexFlatL2 : (98.8499984741211, '0.8810124397277832 SECONDS')
IndexIVFFlat : (98.81999969482422, '0.41861629486083984 SECONDS')
IndexHNSWFlat : (98.8499984741211, '0.5943887233734131 SECONDS')
IndexLSH : (87.05000305175781, '0.948662281036377 SECONDS')


# WITHOUT FAISS
Accuracy : 99.67%   ||     Time : 7.93 SECONDS